In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense
import tensorflow
import datetime



In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [5]:
df.head(1)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.0,1,1,1,101348.88,1


In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])

# one-hot-encoding
ohe_encoder = OneHotEncoder()
encoded_geo = ohe_encoder.fit_transform(df[['Geography']]).toarray()

# concatinating encoded_geo with original df
new_df = pd.concat([df, pd.DataFrame(encoded_geo, columns=ohe_encoder.get_feature_names_out())], axis=1)

In [7]:
# dropping original geography column
new_df.drop(columns='Geography', inplace=True)

In [8]:
# Independent features 
x = new_df.drop(columns='Exited')

# dependent feature
y = new_df['Exited']

In [9]:
# Splitiing data into train and test dataset
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=43)

In [10]:
# Feature Scaling 
scalar = StandardScaler()
x_train = scalar.fit_transform(x_train)
x_test = scalar.transform(x_test)

In [11]:
# creating ANN model 
model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [12]:
# intialising optimizer, loss
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [13]:
# compiling the model 
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [14]:
# creating a log directory
log_dir = 'log/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

In [15]:
# creating tensorboard callbacks
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
tensorboard_callbacks = TensorBoard(log_dir=log_dir, histogram_freq=1)

# creating earlystop callback
early_stop_callback = EarlyStopping(patience=15,monitor='val_loss', restore_best_weights=True)

In [16]:
history = model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=100, callbacks=[tensorboard_callbacks, early_stop_callback]  )

Epoch 1/100


250/250 [==============================] - 3s 5ms/step - loss: 0.3941 - accuracy: 0.8345 - val_loss: 0.3656 - val_accuracy: 0.8565
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3538 - accuracy: 0.8534 - val_loss: 0.3608 - val_accuracy: 0.8560
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3495 - accuracy: 0.8551 - val_loss: 0.3556 - val_accuracy: 0.8550
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3394 - accuracy: 0.8584 - val_loss: 0.3551 - val_accuracy: 0.8535
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3383 - accuracy: 0.8620 - val_loss: 0.3534 - val_accuracy: 0.8565
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3361 - accuracy: 0.8605 - val_loss: 0.3508 - val_accuracy: 0.8560
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3310 - accuracy: 0.8618 - val_loss: 0.3576 - val_accuracy: 0.85

In [17]:
model.save('model.h5')

c:\Users\pandh\OneDrive\Desktop\DeepLearning\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
%reload_ext tensorboard

%tensorboard --logdir=log/fit --port=4

Reusing TensorBoard on port 4 (pid 18136), started 8:48:05 ago. (Use '!kill 18136' to kill it.)

In [19]:
import pickle
with open('label_encoder_gen.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

with open('ohe_encoder.pkl', 'wb') as file:
    pickle.dump(ohe_encoder, file)

with open('scalar.pkl', 'wb') as file:
    pickle.dump(scalar, file)